In [1]:
from groq import Groq
import os 
import json
import sqlparse
from IPython.display import Markdown
import duckdb
import glob
import yaml

In [8]:
client = Groq(api_key = os.getenv('GROQ_API_KEY'))
model = 'llama-3.3-70b-versatile'



In [9]:
system_prompt = '''
You are Groq Advisor, and you are tasked with generating SQL queries for DuckDB based on user questions about data stored in two tables derived from CSV files:

Table: employees.csv
Columns:
employee_id (INTEGER): A unique identifier for each employee.
name (VARCHAR): The full name of the employee.
email (VARCHAR): employee's email address

Table: purchases.csv
Columns:
purchase_id (INTEGER): A unique identifier for each purchase.
purchase_date (DATE): Date of purchase
employee_id (INTEGER): References the employee_id from the employees table, indicating which employee made the purchase.
amount (FLOAT): The monetary value of the purchase.
product_name (STRING): The name of the product purchased

Given a user's question about this data, write a valid DuckDB SQL query that accurately extracts or calculates the requested information from these tables and adheres to SQL best practices for DuckDB, optimizing for readability and performance where applicable.

Here are some tips for writing DuckDB queries:
* DuckDB syntax requires querying from the .csv file itself, i.e. employees.csv and purchases.csv. For example: SELECT * FROM employees.csv as employees
* All tables referenced MUST be aliased
* DuckDB does not implicitly include a GROUP BY clause
* CURRENT_DATE gets today's date
* Aggregated fields like COUNT(*) must be appropriately named

And some rules for querying the dataset:
* Never include employee_id in the output - show employee name instead

Also note that:
* Valid values for product_name include 'Tesla','iPhone' and 'Humane pin'


Question:
--------
{user_question}
--------
Reminder: Generate a DuckDB SQL to answer to the question:
* respond as a valid JSON Document
* [Best] If the question can be answered with the available tables: {"sql": <sql here>}
* If the question cannot be answered with the available tables: {"error": <explanation here>}
* Ensure that the entire output is returned on only one single line
* Keep your query as simple and straightforward as possible; do not use subqueries
'''

In [10]:
def text_to_sql(client,system_prompt,user_question,model):

    completion = client.chat.completions.create(
        model = model,
        response_format = {"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_question
            }
        ]
    )
  
    return completion.choices[0].message.content

In [11]:
def execute_duckdb_query(query):
    original_cwd = os.getcwd()
    os.chdir('data')
    
    try:
        conn = duckdb.connect(database=':memory:', read_only=False)
        query_result = conn.execute(query).fetchdf().reset_index(drop=True)
    finally:
        os.chdir(original_cwd)


    return query_result

In [12]:
user_question = 'What are the most recent purchases?'


llm_response = text_to_sql(client,system_prompt,user_question,model)
sql_json = json.loads(llm_response)
parsed_sql = sqlparse.format(sql_json['sql'], reindent=True, keyword_case='upper')
formatted_sql = f"```sql\n{parsed_sql.strip()}\n```"
display(Markdown(formatted_sql)) 

execute_duckdb_query(parsed_sql)

```sql
SELECT DISTINCT e.name,
                p.purchase_date,
                p.amount,
                p.product_name
FROM purchases.csv AS p
INNER JOIN employees.csv AS e ON p.employee_id = e.employee_id
ORDER BY p.purchase_date DESC
```

,name,purchase_date,amount,product_name
0,Jared Dunn,2024-02-05,75000,Tesla
1,Bertram Gilfoyle,2024-02-04,700,iPhone
2,Dinesh Chugtai,2024-02-03,500,Humane pin
3,Erlich Bachman,2024-02-02,70000,Tesla
4,Richard Hendricks,2024-02-01,750,iPhone
